# Crawling & Pre-processing Twitter data

## Download

In [ ]:
# -*- coding: utf-8 -*-
import tweepy as tw
import sys,jsonpickle
 
consumer_key = ''
consumer_secret = ''
 
qry='jogja'
maxTweets = 30
tweetsPerQry = 30
fName='tweepy ' +qry+ '.json'
 
auth = tw.AppAuthHandler(consumer_key,consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    sys.exit('Authentication failed')
 
sinceId=None;max_id=-1;tweetCount=0
print("Start downloading... {0} tweets".format(maxTweets))
with open(fName,'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,result_type="popular")
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,since_id=sinceId,result_type="popular")
            else:
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1),result_type="popular")
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1),since_id=sinceId,result_type="popular")
            if not new_tweets:
                print('No more Tweets found with Query="{0}"'.format(qry));break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
            tweetCount+=len(new_tweets)
            sys.stdout.write("\r");sys.stdout.write("Tweets has been saved: %.0f" %tweetCount);sys.stdout.flush()
            max_id=new_tweets[-1].id
        except tw.TweepError as e:
            print("some error : " + str(e));break # Aya error, keluar
print ('\nDone! {0} tweets saved at "{1}"'.format(tweetCount,fName))

## Preprocess

In [ ]:
# -*- coding: utf-8 -*-
import json,re
file = open('tweepy ' +qry+ '.json') 
lines = file.readlines()

hasil=open("tweepy_preprocessed.csv",'w')
for i in lines:
    tweet = json.loads(i.strip())['text'].lower()
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*','', tweet).replace('\n', ' ')
    hasil.write(tweet.encode('utf-8')+'\n')
    
hasil.close()

## Labeling

### Load Data

In [ ]:
import json,re
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict

In [ ]:
import pandas as pd

In [ ]:
tweets = pd.read_csv("Book7.csv", sep=';', header=None, names=["text","label"])

In [ ]:
tweets

In [ ]:
tweets.label.value_counts()

In [ ]:
X = tweets.text
y = tweets.label
print(X.shape)
print(y.shape)

## Split Train Test

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()

In [ ]:
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)

In [ ]:
X_test_dtm = vect.transform(X_test)

## NBC

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
%time nb.fit(X_train_dtm, y_train)

In [ ]:
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
y_pred_class

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
metrics.confusion_matrix(y_test, y_pred_class)

# Clustering

## KMeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [ ]:
documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

In [ ]:
labels = [1,1,0,0,1,1,0,0]

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [ ]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

In [ ]:
Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print("Prediction", prediction)

In [ ]:
Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print("Prediction", prediction)

In [ ]:
Y = vectorizer.transform(["photo restaurant mcdonald"])
prediction = model.predict(Y)
print("Prediction", prediction)

In [ ]:
Y = vectorizer.transform(["take your photo off the wall"])
prediction = model.predict(Y)
print("Prediction", prediction)

In [ ]:
# from sklearn import metrics
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, model.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, model.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, model.labels_))
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(labels, model.labels_))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, model.labels_, sample_size=1000))

### Plotlib

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
centroids = model.cluster_centers_

In [ ]:
tsne_init = 'pca'
tsne_perplexity = 20.0
tsne_early_exaggeration = 4.0
tsne_learning_rate = 1000
random_state = 1
model = TSNE(n_components=2, random_state=random_state, init=tsne_init, perplexity=tsne_perplexity,
         early_exaggeration=tsne_early_exaggeration, learning_rate=tsne_learning_rate)

transformed_centroids = model.fit_transform(centroids)
print(transformed_centroids)
plt.scatter(transformed_centroids[:, 0], transformed_centroids[:, 1], marker='x')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);